# Limpieza de Datos del Dataset IGAC

Este notebook procesa el dataset completo aplicando funciones de limpieza.

## Objetivos
1. Procesar el dataset completo en chunks
2. Limpiar nombres de municipios y departamentos
3. Validar y corregir fechas
4. Limpiar valores numéricos
5. Manejar valores nulos
6. Eliminar duplicados
7. Guardar datos limpios en formato Parquet

In [1]:
# Importar librerías
import sys
sys.path.append('../src')


import polars as pl
from pathlib import Path
from tqdm import tqdm
from etl.data_loader import load_full_dataset_lazy, save_to_parquet
from etl.data_cleaner import apply_all_cleaning
from utils.config import CLEANED_PARQUET_FILE

print("Librerías importadas correctamente")

Librerías importadas correctamente


## 1. Carga del Dataset Completo (Lazy Loading)

In [2]:
# Cargar dataset en modo lazy (no se carga en memoria hasta ejecutar operaciones)
lf = load_full_dataset_lazy()
print("Dataset cargado en modo lazy")
print(f"Columnas: {lf.columns}")

Cargando dataset en modo lazy (sin cargar en memoria)...
✓ Dataset cargado en modo lazy
Dataset cargado en modo lazy
Columnas: ['PK', 'MATRICULA', 'FECHA_RADICA_TEXTO', 'FECHA_APERTURA_TEXTO', 'YEAR_RADICA', 'ORIP', 'DIVIPOLA', 'DEPARTAMENTO', 'MUNICIPIO', 'TIPO_PREDIO_ZONA', 'CATEGORIA_RURALIDAD', 'NUM_ANOTACION', 'ESTADO_FOLIO', 'FOLIOS_DERIVADOS', 'Dinámica_Inmobiliaria', 'COD_NATUJUR', 'NOMBRE_NATUJUR', 'NUMERO_CATASTRAL', 'NUMERO_CATASTRAL_ANTIGUO', 'DOCUMENTO_JUSTIFICATIVO', 'COUNT_A', 'COUNT_DE', 'PREDIOS_NUEVOS', 'TIENE_VALOR', 'TIENE_MAS_DE_UN_VALOR', 'VALOR']


C:\Users\carlo\AppData\Local\Temp\ipykernel_8108\4052329036.py:4: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  print(f"Columnas: {lf.columns}")


## 2. Aplicar Limpieza

Aplicamos todas las funciones de limpieza definidas en el módulo `data_cleaner`.

In [3]:
# Recolectar el LazyFrame a DataFrame y aplicar limpieza
print("Ejecutando limpieza del dataset completo...")
print("ADVERTENCIA: Este proceso puede tomar varios minutos debido al tamaño del dataset.\n")

# Collect lazy frame
df = lf.collect()
print(f"Dataset cargado en memoria: {df.shape[0]:,} filas")

# Aplicar limpieza
df_cleaned = apply_all_cleaning(df)
print(f"\nDataset limpio: {df_cleaned.shape[0]:,} filas")

df_cleaned = apply_all_cleaning(df)

Ejecutando limpieza del dataset completo...
ADVERTENCIA: Este proceso puede tomar varios minutos debido al tamaño del dataset.

Dataset cargado en memoria: 30,903,248 filas

🚀 INICIANDO PROCESO COMPLETO DE LIMPIEZA
🧹 LIMPIANDO COLUMNAS STRING...
   ➤ Columnas detectadas: ['MATRICULA', 'ORIP', 'DIVIPOLA', 'TIPO_PREDIO_ZONA', 'CATEGORIA_RURALIDAD', 'ESTADO_FOLIO', 'NOMBRE_NATUJUR']
✓ Columnas string limpiadas
→ Normalizando nombres de municipios...
Normalizando nombres de departamentos...
✓ Departamentos normalizados
Parseando columnas de fecha...
✓ Fechas parseadas
Limpiando valores numéricos...
✓ Valores numéricos limpios
Manejando valores nulos: estrategia report
Eliminando duplicados...
✓ Eliminados 0 duplicados
✅ LIMPIEZA COMPLETA FINALIZADA

Dataset limpio: 30,903,248 filas

🚀 INICIANDO PROCESO COMPLETO DE LIMPIEZA
🧹 LIMPIANDO COLUMNAS STRING...
   ➤ Columnas detectadas: ['MATRICULA', 'ORIP', 'DIVIPOLA', 'TIPO_PREDIO_ZONA', 'CATEGORIA_RURALIDAD', 'ESTADO_FOLIO', 'NOMBRE_NATUJUR']
✓

## 3. Verificar Resultados de Limpieza

In [4]:
# Comparar antes y después
print("\n" + "="*60)
print("COMPARACIÓN ANTES/DESPUÉS DE LIMPIEZA")
print("="*60)
print(f"\nFilas originales: {df.shape[0]:,}")
print(f"Filas después de limpieza: {df_cleaned.shape[0]:,}")
print(f"Filas eliminadas: {df.shape[0] - df_cleaned.shape[0]:,}")
print(f"Porcentaje retenido: {(df_cleaned.shape[0] / df.shape[0]) * 100:.2f}%")


COMPARACIÓN ANTES/DESPUÉS DE LIMPIEZA

Filas originales: 30,903,248
Filas después de limpieza: 30,903,248
Filas eliminadas: 0
Porcentaje retenido: 100.00%


In [5]:
# Ver muestra de datos limpios
print("\nMuestra de datos limpios:")
df_cleaned.head(10)


Muestra de datos limpios:


PK,MATRICULA,FECHA_RADICA_TEXTO,FECHA_APERTURA_TEXTO,YEAR_RADICA,ORIP,DIVIPOLA,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,NUM_ANOTACION,ESTADO_FOLIO,FOLIOS_DERIVADOS,Dinámica_Inmobiliaria,COD_NATUJUR,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,COUNT_A,COUNT_DE,PREDIOS_NUEVOS,TIENE_VALOR,TIENE_MAS_DE_UN_VALOR,VALOR
str,str,datetime[μs],datetime[μs],f64,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,f64,f64,f64,f64,f64,f64
"""63001-280-172322-00006-00843-2…","""280-172322""",2023-07-06 00:00:00,null,2023.0,"""280""","""63001""","""QUINDÍO""","""ARMENIA""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",6,"""ACTIVO""",null,0,843,"""CANCELACIÓN POR VOLUNTAD DE LA…",null,null,"""00-CERTIFICADO 8219 DE 05/07/2…",2.0,2.0,0.0,1.0,0.0,0.0
"""05607-017-047514-00007-00125-2…","""017-47514""",null,null,2018.0,"""17""","""5607""","""ANTIOQUIA""","""RETIRO""","""URBANO""","""INTERMEDIO""",7,"""ACTIVO""",null,1,125,"""COMPRAVENTA""",null,null,"""00-ESCRITURA 74 DE 15/01/2018;…",1.0,1.0,0.0,1.0,0.0,null
"""05615-020-071756-00026-00331-2…","""020-71756""",2021-07-29 00:00:00,2005-08-26 00:00:00,2021.0,"""20""","""5615""","""ANTIOQUIA""","""RIONEGRO""","""RURAL""","""CIUDADES Y AGLOMERACIONES""",26,null,null,1,331,"""REFORMA REGLAMENTO DE PROPIEDA…",null,null,"""01-ESCRITURA 1705 DE 08-07-202…",0.0,1.0,0.0,0.0,0.0,null
"""76001-370-387242-00004-00842-2…","""370-387242""",2020-12-22 00:00:00,1992-05-04 00:00:00,2020.0,"""370""","""76001""","""VALLE DEL CAUCA""","""SANTIAGO DE CALI""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",4,null,null,0,842,"""CANCELACIÓN PROVIDENCIA ADMINI…",null,null,"""02-CERTIFICADO 9200864456 DE 2…",0.0,1.0,0.0,0.0,0.0,null
"""68689-320-023786-00003-00918-2…","""320-23786""",2019-07-17 00:00:00,null,2019.0,"""320""","""68689""","""SANTANDER""","""SAN VICENTE DE CHUCURI""","""RURAL""","""RURAL""",3,"""ACTIVO""",null,0,918,"""DIVISIÓN MATERIAL""",null,null,"""00-ESCRITURA 365 DE 31/05/2019…",1.0,0.0,0.0,1.0,0.0,0.0
"""08001-040-599438-00002-00125-2…","""040-599438""",2020-02-11 00:00:00,2019-12-04 00:00:00,2020.0,"""40""","""8001""","""ATLÁNTICO""","""BARRANQUILLA""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",2,null,null,1,125,"""COMPRAVENTA""",null,null,"""01-ESCRITURA 3341 DE 27-12-201…",1.0,1.0,0.0,0.0,0.0,null
"""70001-340-109226-00003-00109-2…","""340-109226""",2019-08-05 00:00:00,null,2019.0,"""340""","""70001""","""SUCRE""","""SINCELEJO""","""URBANO""","""CIUDADES Y AGLOMERACIONES""",3,"""ACTIVO""",null,1,109,"""ADJUDICACIÓN EN SUCESION""",null,null,"""00-SENTENCIA . DE 27/06/2018; …",8.0,1.0,0.0,1.0,0.0,null
"""68684-312-031475-00001-00918-2…","""312-31475""",2023-01-17 00:00:00,null,2023.0,"""312""","""68684""","""SANTANDER""","""SAN JOSE DE MIRANDA""","""RURAL""","""RURAL""",1,"""ACTIVO""",null,0,918,"""DIVISIÓN MATERIAL""",null,null,"""00-ESCRITURA 727 DE 02/09/2022…",2.0,0.0,1.0,1.0,0.0,0.0
"""50313-236-068706-00009-00314-2…","""236-68706""",null,2015-07-03 14:03:49,2021.0,"""236""","""50313""","""META""","""GRANADA""","""URBANO""","""INTERMEDIO""",9,"""ACTIVO""",null,0,314,"""CONSTITUCIÓN DE USUFRUCTO""",null,null,"""00-ESCRITURA 1515 DEL 25/06/21…",1.0,1.0,0.0,1.0,0.0,0.0


In [6]:
# Verificar valores nulos en columnas críticas
critical_cols = ['PK', 'MUNICIPIO', 'DEPARTAMENTO', 'YEAR_RADICA', 'VALOR']

print("\nValores nulos en columnas críticas:")
for col in critical_cols:
    if col in df_cleaned.columns:
        null_count = df_cleaned[col].null_count()
        print(f"  {col}: {null_count}")


Valores nulos en columnas críticas:
  PK: 0
  MUNICIPIO: 0
  DEPARTAMENTO: 0
  YEAR_RADICA: 0
  VALOR: 20192052


## 4. Guardar Datos Limpios

In [7]:
# Guardar en formato Parquet (comprimido y eficiente)
save_to_parquet(df_cleaned, CLEANED_PARQUET_FILE, compression='snappy')
print(f"\n✓ Datos limpios guardados en: {CLEANED_PARQUET_FILE}")

Guardando datos en formato Parquet: c:\Users\carlo\Documents\KrozFu\repos\fraud-detection-realestate\notebooks\..\data\processed\igac_cleaned.parquet
✓ Archivo guardado: 2085.92 MB

✓ Datos limpios guardados en: c:\Users\carlo\Documents\KrozFu\repos\fraud-detection-realestate\notebooks\..\data\processed\igac_cleaned.parquet


## 5. Estadísticas Finales

In [8]:
# Estadísticas del dataset limpio
print("\nEstadísticas del dataset limpio:")
df_cleaned.describe()


Estadísticas del dataset limpio:


statistic,PK,MATRICULA,FECHA_RADICA_TEXTO,FECHA_APERTURA_TEXTO,YEAR_RADICA,ORIP,DIVIPOLA,DEPARTAMENTO,MUNICIPIO,TIPO_PREDIO_ZONA,CATEGORIA_RURALIDAD,NUM_ANOTACION,ESTADO_FOLIO,FOLIOS_DERIVADOS,Dinámica_Inmobiliaria,COD_NATUJUR,NOMBRE_NATUJUR,NUMERO_CATASTRAL,NUMERO_CATASTRAL_ANTIGUO,DOCUMENTO_JUSTIFICATIVO,COUNT_A,COUNT_DE,PREDIOS_NUEVOS,TIENE_VALOR,TIENE_MAS_DE_UN_VALOR,VALOR
str,str,str,str,str,f64,str,str,str,str,str,str,f64,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64,f64
"""count""","""30903248""","""30903248""","""19658971""","""16533921""",3.0903248e7,"""25637549""","""30903248""","""30903248""","""30903248""","""30903248""","""30903248""",3.0876576e7,"""17323309""","""301690""",3.0903248e7,3.0814164e7,"""30903248""","""0""","""0""","""30895500""",3.0903139e7,3.0903237e7,3.0903248e7,3.0903248e7,3.0903248e7,1.0711196e7
"""null_count""","""0""","""0""","""11244277""","""14369327""",0.0,"""5265699""","""0""","""0""","""0""","""0""","""0""",26672.0,"""13579939""","""30601558""",0.0,89084.0,"""0""","""30903248""","""30903248""","""7748""",109.0,11.0,0.0,0.0,0.0,2.0192052e7
"""mean""",null,null,"""2019-12-05 12:19:34.501595""","""2009-03-28 21:24:30.876509""",2019.19465,null,null,null,null,null,null,8.252333,null,null,0.563075,452.407913,null,null,null,null,1.223576,1.009665,0.094102,0.589571,0.000033,0.079226
"""std""",null,null,null,null,2.651883,null,null,null,null,null,null,22.806727,null,null,0.496006,312.749276,null,null,null,null,2.315217,1.643787,0.291971,0.491912,0.00577,6.648989
"""min""","""05001-001-000013-00014-00313-2…","""001-1000000""","""2015-01-02 00:00:00""","""1899-12-29 00:00:00""",2015.0,"""1""","""11001""","""AMAZONAS""","""ABEJORRAL""","""RURAL""","""CIUDADES Y AGLOMERACIONES""",1.0,"""ACTIVO""","""002-12624""",0.0,100.0,"""ABSTENERSE DE INSCRIBIR NUEVOS…",null,null,"""00-ACTA # 1 DE 22/11/2017; C…",0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",null,null,"""2017-05-25 00:00:00""","""1998-03-11 09:48:22""",2017.0,null,null,null,null,null,null,3.0,null,null,0.0,197.0,null,null,null,null,1.0,1.0,0.0,0.0,0.0,0.0
"""50%""",null,null,"""2020-01-15 00:00:00""","""2015-09-22 00:00:00""",2019.0,null,null,null,null,null,null,6.0,null,null,1.0,317.0,null,null,null,null,1.0,1.0,0.0,1.0,0.0,0.0
"""75%""",null,null,"""2022-08-05 00:00:00""","""2020-08-26 00:00:00""",2022.0,null,null,null,null,null,null,9.0,null,null,1.0,843.0,null,null,null,null,1.0,1.0,0.0,1.0,0.0,0.0
"""max""","""99773-540-013896-00003-00311-2…","""999-9878""","""2023-12-29 00:00:00""","""2223-10-25 00:00:00""",2023.0,"""999""","""99773""","""VICHADA""","""ZONA BANANERA""","""URBANO""","""RURAL DISPERSO""",999.0,"""CUSTODIA""","""540-9989;540-10001;540-9991;54…",1.0,999.0,"""VALORIZACIÓN""",null,null,"""99-SIN INFORMACION s/n DE 22-1…",994.0,609.0,1.0,1.0,1.0,999.12


In [9]:
# Distribución por año después de limpieza
year_dist = df_cleaned.group_by('YEAR_RADICA').agg([
    pl.count().alias('Cantidad')
]).sort('YEAR_RADICA')

print("\nDistribución por año (datos limpios):")
print(year_dist)


Distribución por año (datos limpios):
shape: (9, 2)
┌─────────────┬──────────┐
│ YEAR_RADICA ┆ Cantidad │
│ ---         ┆ ---      │
│ f64         ┆ u32      │
╞═════════════╪══════════╡
│ 2015.0      ┆ 3138029  │
│ 2016.0      ┆ 3656455  │
│ 2017.0      ┆ 3184604  │
│ 2018.0      ┆ 3020947  │
│ 2019.0      ┆ 3186849  │
│ 2020.0      ┆ 2671398  │
│ 2021.0      ┆ 3850368  │
│ 2022.0      ┆ 4223565  │
│ 2023.0      ┆ 3971033  │
└─────────────┴──────────┘


C:\Users\carlo\AppData\Local\Temp\ipykernel_8108\2993156178.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('Cantidad')


## Conclusiones

- Dataset limpio guardado en formato Parquet
- Valores nulos manejados
- Duplicados eliminados
- Nombres de municipios y departamentos normalizados
- Fechas parseadas correctamente
- Valores numéricos validados

**Próximo paso:** Notebook 03 - Estandarización de datos